In [ ]:
# IMPORT ALL YOUR LIBRARIES
# SUGGESTED LIBRARIES
!pip install accelerate
!pip install transformers
import accelerate
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import TensorDataset
from transformers import TrainingArguments, Trainer
import matplotlib.pyplot as plt

In [2]:
!mkdir /content/data

In [3]:
df = pd.read_csv("/content/augm_data.csv")
# target_values = ['Martyr', 'Instigator', 'Guardian', 'Incompetent','Tyrant','Conspirator','Virtuous']

# for value in target_values:
#     indices_to_drop = df[df['fine_grained_role1'] == value].index[:20]
#     df = df.drop(indices_to_drop)

target_values = ['Martyr', 'Instigator', 'Virtuous']

for value in target_values:
    if value == 'Instigator':
        indices_to_drop = df[df['fine_grained_role1'] == value].index[:30]
    else:
        indices_to_drop = df[df['fine_grained_role1'] == value].index[:20]
    df = df.drop(indices_to_drop)

df.reset_index(drop=True, inplace=True)
result_df = df.copy()
result_df

sentence        label  \
0     Driven by escalating tensions and NATO's poten...   Antagonist   
1     Greta Thunberg, a figurehead of manufactured c...   Antagonist   
2     Zelensky's increasingly desperate pleas for mo...   Antagonist   
3     Zelensky's envoys, desperately trying to sooth...   Antagonist   
4     The Daily Sceptic, a mouthpiece for Kremlin di...   Antagonist   
...                                                 ...          ...   
1699  d our political elite that’s destroying our pl...   Antagonist   
1700  Zelensky Exposed: Former Advisor Reveals How U...   Antagonist   
1701  ould be "bitterly and bloodily resisted" by th...  Protagonist   
1702  Al Gore, a supposed environmental champion, co...   Antagonist   
1703  While the United States debates aid to Ukraine...     Innocent   

     mentioned_entity fine_grained_role1  
0               Putin  Foreign Adversary  
1      Greta Thunberg           Deceiver  
2          Zelensky's        Incompetent  
3              Moscow            Traitor  
4       Daily Sceptic        Conspirator  
...               ...                ...  
1699      No Sale Gun          Terrorist  
1700         Zelensky        Incompetent  
1701               UK           Guardian  
1702          Al Gore           Deceiver  
1703    United States          Forgotten  

[1704 rows x 4 columns]

In [4]:
prompt_lengths = result_df['sentence'].apply(len)

filtered_df = result_df[prompt_lengths >= 500]
result_df = filtered_df.copy()

In [5]:


# Create binary columns for 'label' and 'fine_grained_role1'
# label_binary = pd.get_dummies(result_df['label'])
role_binary = pd.get_dummies(result_df['fine_grained_role1'])
print(result_df['fine_grained_role1'].value_counts())
# Concatenate the binary columns with the original DataFrame
df = pd.concat([result_df, role_binary], axis=1)
df = df.drop(columns=['label'])
df = df.drop(columns=['mentioned_entity'])
df = df.drop(columns=['fine_grained_role1'])
df = df.rename(columns={'sentence': 'Prompt'})
# Display the DataFrame
df.head()

fine_grained_role1
Spy                  62
Bigot                61
Guardian             58
Instigator           58
Conspirator          56
Foreign Adversary    56
Tyrant               56
Martyr               56
Traitor              53
Incompetent          53
Virtuous             53
Saboteur             51
Corrupt              44
Underdog             43
Rebel                43
Peacemaker           39
Victim               39
Exploited            32
Deceiver             30
Scapegoat            23
Terrorist            21
Forgotten            13
Name: count, dtype: int64


Prompt  Bigot  Conspirator  \
0  Driven by escalating tensions and NATO's poten...  False        False   
2  Zelensky's increasingly desperate pleas for mo...  False        False   
3  Zelensky's envoys, desperately trying to sooth...  False        False   
6  Zelensky's envoys are desperately scrambling t...   True        False   
7  Despite the US's powerful military might, the ...  False        False   

   Corrupt  Deceiver  Exploited  Foreign Adversary  Forgotten  Guardian  \
0    False     False      False               True      False     False   
2    False     False      False              False      False     False   
3    False     False      False              False      False     False   
6    False     False      False              False      False     False   
7    False     False      False              False      False     False   

   Incompetent  ...  Rebel  Saboteur  Scapegoat    Spy  Terrorist  Traitor  \
0        False  ...  False     False      False  False      False    False   
2         True  ...  False     False      False  False      False    False   
3        False  ...  False     False      False  False      False     True   
6        False  ...  False     False      False  False      False    False   
7        False  ...  False     False      False  False      False    False   

   Tyrant  Underdog  Victim  Virtuous  
0   False     False   False     False  
2   False     False   False     False  
3   False     False   False     False  
6   False     False   False     False  
7   False      True   False     False  

[5 rows x 23 columns]

In [6]:
bool_columns = df.select_dtypes(include=['bool']).columns
df[bool_columns] = df[bool_columns].astype(int)

In [7]:
df['Prompt'][8]

' \n\n Actress Jane Fonda’s latest performance as a climate change activist drew a slew of criticism as she protested the Alberta oil sands in the Canadian province.\n\nFonda was castigated by locals, ambushed by a social media reporter and criticized by a left-leaning provincial premier. On Wednesday, at an Edmonton, Alberta news conference, she condemned a fellow-progressive, Canadian Prime Minister Justin Trudeauu, saying that “people shouldn’t be fooled by good-looking Liberals,” and accused him of “betraying” the Paris climate agreement.\n\nFonda claimed that, like President Barack Obama, her first impression of Trudeau was one of admiration: “When I heard that your prime minister, the shining hope at the …'

In [8]:
# label_df = result_df[['Prompt','Protagonist','Antagonist','Innocent']]
label_df = df.copy()
label_df

Prompt  Bigot  Conspirator  \
0     Driven by escalating tensions and NATO's poten...      0            0   
2     Zelensky's increasingly desperate pleas for mo...      0            0   
3     Zelensky's envoys, desperately trying to sooth...      0            0   
6     Zelensky's envoys are desperately scrambling t...      1            0   
7     Despite the US's powerful military might, the ...      0            0   
...                                                 ...    ...          ...   
1697  ars are always pricey. He admits that neglecti...      0            0   
1698  Klitschko, undeterred by the recent kidnapping...      0            0   
1699  d our political elite that’s destroying our pl...      0            0   
1700  Zelensky Exposed: Former Advisor Reveals How U...      0            0   
1701  ould be "bitterly and bloodily resisted" by th...      0            0   

      Corrupt  Deceiver  Exploited  Foreign Adversary  Forgotten  Guardian  \
0           0         0          0                  1          0         0   
2           0         0          0                  0          0         0   
3           0         0          0                  0          0         0   
6           0         0          0                  0          0         0   
7           0         0          0                  0          0         0   
...       ...       ...        ...                ...        ...       ...   
1697        0         1          0                  0          0         0   
1698        0         0          0                  0          0         0   
1699        0         0          0                  0          0         0   
1700        0         0          0                  0          0         0   
1701        0         0          0                  0          0         1   

      Incompetent  ...  Rebel  Saboteur  Scapegoat  Spy  Terrorist  Traitor  \
0               0  ...      0         0          0    0          0        0   
2               1  ...      0         0          0    0          0        0   
3               0  ...      0         0          0    0          0        1   
6               0  ...      0         0          0    0          0        0   
7               0  ...      0         0          0    0          0        0   
...           ...  ...    ...       ...        ...  ...        ...      ...   
1697            0  ...      0         0          0    0          0        0   
1698            0  ...      0         0          0    0          0        0   
1699            0  ...      0         0          0    0          1        0   
1700            1  ...      0         0          0    0          0        0   
1701            0  ...      0         0          0    0          0        0   

      Tyrant  Underdog  Victim  Virtuous  
0          0         0       0         0  
2          0         0       0         0  
3          0         0       0         0  
6          0         0       0         0  
7          0         1       0         0  
...      ...       ...     ...       ...  
1697       0         0       0         0  
1698       0         0       0         0  
1699       0         0       0         0  
1700       0         0       0         0  
1701       0         0       0         0  

[1000 rows x 23 columns]

In [9]:
train_df = label_df[:800]
validation_df = label_df[800:950]
test_df = label_df[950:]
train_labels_df = label_df[:800]
validation_labels_df = label_df[800:950]
test_labels_df = label_df[950:]


In [10]:
train_sentences = train_df['Prompt'].values.tolist()
train_labels = train_labels_df.iloc[:, 1:].values
train_labels = train_labels.astype(np.float32)

validation_sentences = validation_df['Prompt'].values.tolist()
validation_labels = validation_labels_df.iloc[:, 1:].values
validation_labels = validation_labels.astype(np.float32)

In [11]:
num_columns = train_labels.shape[1]
print("Number of columns:", num_columns)

Number of columns: 22


In [12]:
# from transformers import AutoTokenizer

# checkpoint = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=22, problem_type="multi_label_classification")
# model

from transformers import BartForSequenceClassification, AutoTokenizer

checkpoint = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = BartForSequenceClassification.from_pretrained(checkpoint, num_labels=22, problem_type="multi_label_classification")
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-large-cnn and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BartForSequenceClassification(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
   

In [13]:
class ArgDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [14]:
train_encodings = tokenizer(train_sentences, padding = True, truncation = True, max_length = 512)
train_dataset = ArgDataset(train_encodings, train_labels)

validation_encodings = tokenizer(validation_sentences, padding = True, truncation = True, max_length = 512)
validation_dataset = ArgDataset(validation_encodings, validation_labels)

In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss

def compute_metrics(pred):
    labels = pred.label_ids
    preds = (pred.predictions[0] > 0.385).astype(int)

    # Compute exact match ratio
    exact_match_ratio = (preds == labels).all(axis=1).mean()

    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted', zero_division=0)
    recall = recall_score(labels, preds, average='weighted', zero_division=0)
    f1 = f1_score(labels, preds, average='weighted', zero_division=0)

    # Generate classification report
    # report = classification_report(labels, preds, target_names=label_names, zero_division=0)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'exact_match_ratio': exact_match_ratio,
        # 'classification_report': report
    }


In [16]:
# import os
# os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import Trainer, TrainingArguments,DefaultFlowCallback

training_args = TrainingArguments(
    output_dir= './results',
    eval_strategy="epoch",
    logging_dir="./logs",
    save_strategy="epoch",
    num_train_epochs = 10,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    load_best_model_at_end=True,
    logging_steps=50,
    metric_for_best_model="exact_match_ratio"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics,
    # callbacks=[DefaultFlowCallback]
)

trainer.train()


In [ ]:
import matplotlib.pyplot as plt

# Extract the log history from the trainer
log_history = trainer.state.log_history

# Initialize lists for storing values
losses = []
accuracies = []
precisions = []
f1_scores = []
recalls = []

# Parse through the log history and extract values
for entry in log_history:
    if 'eval_loss' in entry and 'eval_accuracy' in entry and 'eval_precision' in entry and 'eval_recall' in entry and 'eval_f1' in entry:
        losses.append(entry['eval_loss'])
        accuracies.append(entry['eval_accuracy'])
        precisions.append(entry['eval_precision'])
        f1_scores.append(entry['eval_f1'])
        recalls.append(entry['eval_recall'])

# Define epochs (adjust based on your actual epoch range)
epochs = list(range(1, len(losses) + 1))

# Create a figure and axis for plotting
fig, ax1 = plt.subplots(figsize=(14, 6))

# Plot the first metric (loss) on the primary y-axis
ax1.plot(epochs, losses, color='tab:orange', label='Validation Loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Validation Loss', color='tab:orange')
ax1.tick_params(axis='y', labelcolor='tab:orange')

# Create a secondary y-axis for Accuracy
ax2 = ax1.twinx()
ax2.plot(epochs, accuracies, color='tab:blue', linestyle='--', label='Accuracy')
ax2.set_ylabel('Accuracy', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')

# Create a tertiary y-axis for Precision
ax3 = ax1.twinx()
ax3.spines['right'].set_position(('outward', 60))  # Offset the third y-axis
ax3.plot(epochs, precisions, color='tab:green', linestyle='-', label='Precision')
ax3.set_ylabel('Precision', color='tab:green')
ax3.tick_params(axis='y', labelcolor='tab:green')

# Create a fourth y-axis for Recall
ax4 = ax1.twinx()
ax4.spines['right'].set_position(('outward', 120))  # Offset the fourth y-axis
ax4.plot(epochs, recalls, color='tab:red', linestyle=':', label='Recall')
ax4.set_ylabel('Recall', color='tab:red')
ax4.tick_params(axis='y', labelcolor='tab:red')

# Create a fifth y-axis for F1-score
ax5 = ax1.twinx()
ax5.spines['right'].set_position(('outward', 180))  # Offset the fifth y-axis
ax5.plot(epochs, f1_scores, color='tab:purple', linestyle='-.', label='F1-Score')
ax5.set_ylabel('F1-Score', color='tab:purple')
ax5.tick_params(axis='y', labelcolor='tab:purple')

# Add legends for each line
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines3, labels3 = ax3.get_legend_handles_labels()
lines4, labels4 = ax4.get_legend_handles_labels()
lines5, labels5 = ax5.get_legend_handles_labels()

ax2.legend(lines1 + lines2 + lines3 + lines4 + lines5,
           labels1 + labels2 + labels3 + labels4 + labels5,
           loc='lower left', bbox_to_anchor=(1.05, 1))

# Title and grid
plt.title('Validation Metrics vs Epochs')
plt.grid(False)

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
model.save_pretrained("/content/")
tokenizer.save_pretrained("/content/")
# model = BertForSequenceClassification.from_pretrained("/content/")

In [ ]:
model = BartForSequenceClassification.from_pretrained("/content/", num_labels=22, problem_type="multi_label_classification")
tokenizer = AutoTokenizer.from_pretrained("/content/")

In [ ]:
test_df.insert(0, 'Index', range(1, len(test_labels_df) + 1))

In [ ]:
import pandas as pd
import requests
import numpy as np

# Raw URL for the annotations file
annotations_url = "https://raw.githubusercontent.com/Khubaib2002/DATA_TASK_10/main/subtask-1-entity-mentions.txt"

# Load the annotations file
df = pd.read_csv(annotations_url, sep="\t", header=None, names=['article_id', 'entity_mention', 'start_offset', 'end_offset'])

# Base URL for the raw documents
base_url = "https://raw.githubusercontent.com/Khubaib2002/DATA_TASK_10/main/subtask-1-documents/"

# Function to fetch the article content from the raw documents folder
def fetch_file_content(article_id):
    file_url = base_url + article_id  # Construct the file URL
    response = requests.get(file_url)
    if response.status_code == 200:
        return response.text  # Return the content if the file is found
    else:
        return f"File {article_id} not found"

df['article_content'] = df['article_id'].apply(fetch_file_content)

def extract_contextual_content(row):
    content = row['article_content']
    start = row['start_offset']
    end = row['end_offset']

    if isinstance(content, str) and len(content) >= end:
        # Determine the range for 'before' based on the start position
        begin = max(0,start-250)
        finish = max(len(content),end+250)
        before = content[begin:start]
        after = content[end:finish]
        sentence = before + row['entity_mention'] + after

        return sentence


df['contextual_content'] = df.apply(extract_contextual_content, axis=1)
dedf = df.copy()
dedf.insert(0, 'Index', range(1, len(dedf) + 1))
dedf

In [23]:
import torch
import pandas as pd
from transformers import BartForSequenceClassification, AutoTokenizer

# Load the model and tokenizer
model = BartForSequenceClassification.from_pretrained("/content/", num_labels=22, problem_type="multi_label_classification")
tokenizer = AutoTokenizer.from_pretrained("/content/")

# Define the threshold for multi-label classification
threshold = 0.16
predictions = []

# Assuming dedf contains the data you want to classify
for idx, row in dedf.iterrows():
    arg_id = row['Index']
    text = row['contextual_content']

    # Tokenizing the input text
    encoding = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')

    # Disable gradient computation during inference
    with torch.no_grad():
        outputs = model(**encoding)

    # Apply Sigmoid to get probabilities
    sigmoid = torch.nn.Sigmoid()
    predictions_tensor = sigmoid(outputs.logits)

    # Apply thresholding to get predicted labels
    predicted_labels = (predictions_tensor.flatten() >= threshold).int().tolist()

    # If no labels exceed the threshold, pick the label with the highest probability
    if not any(predicted_labels):
        # Get the index of the label with the highest probability
        highest_prob_index = torch.argmax(predictions_tensor).item()
        predicted_labels = [0] * len(predicted_labels)  # Reset all labels to 0
        predicted_labels[highest_prob_index] = 1  # Set the highest probability label to 1

    # Append the result with the argument ID
    predictions.append([arg_id] + predicted_labels)

# Convert predictions to a DataFrame
data = {'Index': [entry[0] for entry in predictions]}

# Assuming `test_labels_df.columns[2:]` contains your label column names
for i, label in enumerate(test_labels_df.columns[2:]):
    data[label] = [entry[i+1] for entry in predictions]

predictions_df = pd.DataFrame(data)



You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1', '2': 'LABEL_2', '3': 'LABEL_3', '4': 'LABEL_4', '5': 'LABEL_5', '6': 'LABEL_6', '7': 'LABEL_7', '8': 'LABEL_8', '9': 'LABEL_9', '10': 'LABEL_10', '11': 'LABEL_11', '12': 'LABEL_12', '13': 'LABEL_13', '14': 'LABEL_14', '15': 'LABEL_15', '16': 'LABEL_16', '17': 'LABEL_17', '18': 'LABEL_18', '19': 'LABEL_19', '20': 'LABEL_20', '21': 'LABEL_21'}. The number of labels wil be overwritten to 22.


KeyboardInterrupt: 

In [ ]:
predictions_df.to_csv('checker.csv', sep="\t", index=False, header=False)
predictions_df

columns_to_check = predictions_df.columns[1:]
has_single_one = (predictions_df[columns_to_check].sum(axis=1) > 0)

df['has_single_one'] = has_single_one

df[has_single_one].count()


In [ ]:
#predictions_df.valuecounts
#print value counts
df = predictions_df.copy()
df['article_id'] = dedf['article_id']
df = df.drop(columns=['Index'])

fine_grained_columns = df.columns.difference(['article_ID', 'Protagonist' , 'Antagonist',	'Innocent'])
df['fine_grained'] = df[fine_grained_columns].apply(lambda row: '; '.join(row.index[row == 1]), axis=1)

df = df.drop(columns=fine_grained_columns)

df.insert(0, 'article_id', dedf['article_id'])
df.insert(1, 'entity_mention', dedf['entity_mention'])
df.insert(2, 'start_offset', dedf['start_offset'])
df.insert(3, 'end_offset', dedf['end_offset'])
# df.insert(4, 'label', 'Antagonist')

split_columns = df['fine_grained'].str.split(';', expand=True)

split_columns = split_columns.fillna('')

df[['fine_grained_1', 'fine_grained_2']] = split_columns.iloc[:, :2]
df = df.drop(columns=['fine_grained'])
df = df.drop(columns=['fine_grained_2'])


import pandas as pd

# Sample dataframe (assuming 'df' is your dataframe)
# Create a dictionary to map fine-grained roles to their labels
role_to_label = {
    'Guardian': 'Protagonist',
    'Martyr': 'Protagonist',
    'Peacemaker': 'Protagonist',
    'Rebel': 'Protagonist',
    'Underdog': 'Protagonist',
    'Virtuous': 'Protagonist',
    'Instigator': 'Antagonist',
    'Conspirator': 'Antagonist',
    'Tyrant': 'Antagonist',
    'Foreign Adversary': 'Antagonist',
    'Traitor': 'Antagonist',
    'Spy': 'Antagonist',
    'Saboteur': 'Antagonist',
    'Corrupt': 'Antagonist',
    'Incompetent': 'Antagonist',
    'Terrorist': 'Antagonist',
    'Deceiver': 'Antagonist',
    'Bigot': 'Antagonist',
    'Forgotten': 'Innocent',
    'Exploited': 'Innocent',
    'Victim': 'Innocent',
    'Scapegoat': 'Innocent'
}

# Function to map fine_grained role to the correct label
def assign_label(fine_grained_value):
    roles = fine_grained_value.split(';')  # Split multiple roles by semicolon
    # For each role, assign its corresponding label
    labels = [role_to_label.get(role.strip(), 'Unknown') for role in roles]
    return labels

# df['labels'] = df['fine_grained_1'].apply(assign_label)

df.insert(4, 'labels', df['fine_grained_1'].apply(assign_label))

df['labels'] = df['labels'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)


# df.to_csv('label', sep='\t', index=False
# df[['fine_grained_1', 'fine_grained_2']] = df['fine_grained'].str.split(';', expand=True)

# Check the result
print(df.head())

# 5. Save to a tab-separated text file
df.to_csv('outputers6.txt', sep='\t', index=False)

# Display the result
df
